In [1]:
# Definición de variables 
import pandapower.networks as pp_net
import pandapower as pp
# cargamos el modelo eléctrico 
net = pp_net.case39()
#net.sn_mva = 1.0
pp.runpp(net)
net

This pandapower network includes the following parameter tables:
   - bus (39 elements)
   - load (21 elements)
   - gen (9 elements)
   - ext_grid (1 element)
   - line (35 elements)
   - trafo (11 elements)
   - poly_cost (10 elements)
   - bus_geodata (39 elements)
 and the following results tables:
   - res_bus (39 elements)
   - res_line (35 elements)
   - res_trafo (11 elements)
   - res_ext_grid (1 element)
   - res_load (21 elements)
   - res_gen (9 elements)

In [6]:
list(net.line.index.values)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34]

In [5]:
from pyomo.environ import *

model = ConcreteModel()
model.i = Set(initialize=[1,2,3,4], doc='Buses')
model.j = Set(initialize=[1,2,3,4], doc='Buses')
model.c = Set(initialize=[1,2,3], doc='Lines')
model.t = Set(initialize=[1,2,3,4,5], doc='Time')

model.V_LineP = Var(model.i, model.j, model.c, model.t, within=NonNegativeReals, doc='Real power flowing from bus i towards bus j on line c at time t')


In [21]:
model.Nodes = Set()

In [ ]:
model.Arcs = Set(within=model.Nodes*model.Nodes)

In [19]:
model.Arcs.display()

Arcs : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain      : Size : Members
    None :     2 : Arcs_domain :    0 :      {}


In [7]:
branchstatus = dict(
    ((net.line.iloc[c]['from_bus'],net.line.iloc[c]['to_bus'],c), 
    net.line.iloc[c]['in_service']) 
    for c in range(net.line.shape[0])
    )

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.x = Var(model.i, model.j, model.c,model.t, within=Binary)

model.V_LineP = Var(model.i, model.j, model.c,model.t, bounds=(0,1000),doc='Real power flowing from bus i towards bus j on line c at time t')

# Set the allowed combinations
allowed_combinations = [ (0,1,2,3), (1,2,3,4), (4,5,6,7) ]

def V_LineP_restriction(model, i, j, c, t):
    if (i,j,c,t) in allowed_combinations:
        return model.x[i,j,c,t]*model.V_LineP[i,j,c,t] <=1000
    else:
        return Constraint.Skip

model.V_LineP_restriction = Constraint(model.i, model.j, model.c,model.t, rule=V_LineP_restriction)

def only_one_active(model):
    return sum(model.x[i,j,c,t] for i,j,c,t in allowed_combinations) <= 1

model.only_one_active = Constraint(rule=only_one_active)
